# EDA

## Carga de librerías necesarias

In [1]:
from ast import literal_eval

import pandas as pd

## DataFrame de los canales

Cada fila representa un canal.

In [2]:
data_path = '../data/_META_collection.json'
df = pd.read_json(data_path)
print(df.shape)
df.head().iloc[:, 1:]

(404, 6)


,total_documents,total_cleaned_texts,language_counts,topic_metrics,topics
0,6139,2496,"[{'_id': 'SPANISH', 'count': 4495}, {'_id': 'P...","{'similarity': 0.36375413669217904, 'noise': 0...","[{'tid': 0, 'n_messages': 111, 'description': ..."
1,4622,3686,"[{'_id': 'ENGLISH', 'count': 4241}, {'_id': 'L...","{'similarity': 0.369337386184171, 'noise': 0.4...","[{'tid': 0, 'n_messages': 171, 'description': ..."
2,479,264,"[{'_id': 'SPANISH', 'count': 327}, {'_id': 'EN...","{'similarity': 0.212316362834281, 'noise': 0.5...","[{'tid': 0, 'n_messages': 66, 'description': '..."
3,157,150,"[{'_id': 'ENGLISH', 'count': 155}]","{'similarity': 0.187268570707492, 'noise': 0.1...","[{'tid': 0, 'n_messages': 100, 'description': ..."
4,2895,2605,"[{'_id': 'LITHUANIAN', 'count': 2863}, {'_id':...","{'similarity': 0.375353056952453, 'noise': 0.6...","[{'tid': 0, 'n_messages': 123, 'description': ..."


## DataFrame de idiomas

In [3]:
df_languages = df[["_id", "language_counts"]].rename(columns={"_id": "channel_id"})
df_languages = df_languages.explode("language_counts")
df_languages.head().iloc[:, 1:]

,language_counts
0,"{'_id': 'SPANISH', 'count': 4495}"
0,"{'_id': 'PORTUGUESE', 'count': 49}"
0,"{'_id': 'ENGLISH', 'count': 38}"
0,"{'_id': 'LATIN', 'count': 23}"
0,"{'_id': 'ITALIAN', 'count': 14}"


In [4]:
df_languages["language"] = df_languages["language_counts"].apply(lambda x: x["_id"])
df_languages["count"] = df_languages["language_counts"].apply(lambda x: x["count"])
df_languages = df_languages.rename(columns={"channel_id": "_id"})
df_languages = df_languages.drop(columns=["language_counts"]).reset_index(drop=True)
df_languages.head().iloc[:, 1:]

,language,count
0,SPANISH,4495
1,PORTUGUESE,49
2,ENGLISH,38
3,LATIN,23
4,ITALIAN,14


In [5]:
print(f"{df_languages.language.nunique()} idiomas diferentes")

73 idiomas diferentes


In [6]:
# Top 10 idiomas
df_languages.groupby(["language"]).sum("count").sort_values("count", ascending=False).head(10)

,count
language,
ENGLISH,2318202
TURKISH,1445421
SPANISH,1299807
LITHUANIAN,246815
FRENCH,102690
GERMAN,85713
RUSSIAN,46087
PORTUGUESE,22262
LATIN,18860


In [7]:
# Encontrar el idioma principal por canal (mayor 'count')
idx = df_languages.groupby("_id")["count"].idxmax()
df_primary = df_languages.loc[idx].reset_index(drop=True)
df_primary.head().iloc[:, 1:]

,language,count
0,ENGLISH,325
1,ENGLISH,301
2,ENGLISH,1956
3,ENGLISH,9253
4,LITHUANIAN,2700


In [8]:
for lang in ["ENGLISH", "TURKISH", "SPANISH"]:
  # Filtrar los canales cuyo idioma principal es Inglés, Turco o Español
  filtered_channels = df_primary[df_primary["language"].isin([lang])]

  # Calcular el porcentaje de canales cuyo idioma principal es el especificado
  total_channels = len(df_primary)
  percentage = (len(filtered_channels) / total_channels) * 100
  print(f"Porcentaje de canales cuyo idioma principal es {lang}: {percentage:.2f}%")

Porcentaje de canales cuyo idioma principal es ENGLISH: 48.02%
Porcentaje de canales cuyo idioma principal es TURKISH: 21.78%
Porcentaje de canales cuyo idioma principal es SPANISH: 18.07%


## DataFrame de métricas sobre los tópicos

In [9]:
df_topic_metrics = df.loc[:, ["_id", "topic_metrics"]]

df_topic_metrics["similarity"] = df_topic_metrics["topic_metrics"].apply(lambda x: x["similarity"])
df_topic_metrics["noise"] = df_topic_metrics["topic_metrics"].apply(lambda x: x["noise"])
df_topic_metrics["n_topics"] = df_topic_metrics["topic_metrics"].apply(lambda x: x["n_topics"])
df_topic_metrics["topic_sizes"] = df_topic_metrics["topic_metrics"].apply(lambda x: x["topic_sizes"])
df_topic_metrics["n_valid_topics"] = df_topic_metrics["topic_metrics"].apply(lambda x: x["n_valid_topics"])

df_topic_metrics = df_topic_metrics.drop(columns=["topic_metrics"])
df_topic_metrics.head().iloc[:, 1:]

,similarity,noise,n_topics,topic_sizes,n_valid_topics
0,0.363754,0.490385,26,"[111, 106, 85, 65, 63, 60, 59, 58, 55, 54, 54,...",1
1,0.369337,0.462561,35,"[171, 163, 143, 137, 133, 113, 96, 95, 79, 58,...",1
2,0.212316,0.503788,2,"[66, 65]",1
3,0.187269,0.120000,2,"[100, 32]",1
4,0.375353,0.651056,19,"[123, 100, 88, 76, 68, 54, 42, 41, 38, 35, 33,...",1


In [10]:
df_topic_metrics["topic_sizes"].explode().astype(int).describe().to_frame().T

,count,mean,std,min,25%,50%,75%,max
topic_sizes,21614.0,57.30929,48.198698,20.0,28.0,41.0,68.0,821.0


In [11]:
df_topic_metrics.shape

(404, 6)

In [12]:
df_topic_metrics.describe()

,similarity,noise,n_topics,n_valid_topics
count,404.000000,404.000000,404.000000,404.0
mean,0.337022,0.549267,53.500000,1.0
std,0.037360,0.108946,76.970024,0.0
min,0.166757,0.028571,2.000000,1.0
25%,0.321768,0.510039,10.000000,1.0
50%,0.344490,0.569541,27.000000,1.0
75%,0.362218,0.618329,65.000000,1.0
max,0.414412,0.775210,713.000000,1.0


## DataFrame de tópicos

In [13]:
df_topics = df.loc[:, ["_id", "topics"]]
df_topics = df_topics.explode("topics")
df_topics.head().iloc[:, 1:]

,topics
0,"{'tid': 0, 'n_messages': 111, 'description': '..."
0,"{'tid': 1, 'n_messages': 106, 'description': '..."
0,"{'tid': 2, 'n_messages': 85, 'description': '2..."
0,"{'tid': 3, 'n_messages': 65, 'description': '3..."
0,"{'tid': 4, 'n_messages': 63, 'description': '4..."


In [14]:
df_topics["tid"] = df_topics["topics"].apply(lambda x: x["tid"])
df_topics["n_messages"] = df_topics["topics"].apply(lambda x: x["n_messages"])
df_topics["description"] = df_topics["topics"].apply(lambda x: x["description"])
df_topics["keywords"] = df_topics["topics"].apply(lambda x: x["keywords"])
df_topics["representative_docs"] = df_topics["topics"].apply(lambda x: x["representative_docs"])
df_topics["theme"] = df_topics["topics"].apply(lambda x: x["theme"])

df_topics = df_topics.drop(columns=["topics"]).reset_index(drop=True)
df_topics.head().iloc[:, 1:]

,tid,n_messages,description,keywords,representative_docs,theme
0,0,111,0_vacunamos_novavax_fauci_reacciones,"[vacunamos, novavax, fauci, reacciones, todavia]",[estracto de la agencia europea del medicament...,Conspiracy theories and misinformation about C...
1,1,106,1_encontrado_oficialialistas_mujer_desconectado,"[encontrado, oficialialistas, mujer, desconect...","[los viajes espaciales, la creacion de una pan...",The theme revolves around conspiracy theories ...
2,2,85,2_expreso0demedianoche_directo_anonimo_descarg...,"[expreso0demedianoche, directo, anonimo, desca...",[el expreso de media noche directo del 24/02 /...,Unauthorized use of a popular podcast's name a...
3,3,65,3_despiertos_actuamos_inocencia_hijos,"[despiertos, actuamos, inocencia, hijos, egoce...",[da mucho asco ver ciertos personajes que van ...,"Critique of egoism, superficiality, and lack o..."
4,4,63,4_protestas_republica_belga_cerrados,"[protestas, republica, belga, cerrados, vacuna...",[la gente se defiende: suiza fuerza un referé...,Public discontent and resistance against gover...


In [15]:
df_topics.shape

(21614, 7)

In [16]:
df_topics.to_csv("../data/topics.csv", index=False)